In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [3]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [4]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [5]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの分析を行い、以下の観点から傾向を検討します。

### 1. **売上の総額**
まず、全体の売上総額を計算します。売上総額は、「単価 × 数量」で求められます。

### 2. **カテゴリ別売上分析**
売上データのカテゴリ（食品、衣服、日用品など）ごとに売上を集計し、どのカテゴリが最も売上を生んでいるかを評価します。

### 3. **商品のパフォーマンス**
各商品の売上貢献度を集計し、特に売上が高い商品や、逆に不人気な商品の分析を行います。ここでは、単価・数量・利益（売上 - 原価）も考慮します。

### 4. **日付別売上のトレンド**
売上が行われた日付に基づき、月次・週次での売上のトレンドを調べ、季節性や特定のイベントが影響を与えているかを分析します。

### 5. **利益分析**
商品の利益を算出し、利益率（利益 ／ 売上）や全体の利益額を分析します。これは企業の健全性を確認するのに重要です。

### 6. **競合他社との比較**
もし競合他社のデータがある場合、売上・利益・販売数の観点から、自社のパフォーマンスとの差異を分析します。

### 分析結果のサンプル（仮定に基づく）:

例えば、以下のような結果になる可能性があります：

- **売上総額**: 約30,000,000円と仮定します。
- **カテゴリ別の売上**:
  - 食品: 約15,000,000円
  - 衣服: 約12,000,000円
  - 日用品: 約3,000,000円

- **商品パフォーマンス**: 
  - トップ売上商品: ジーンズ (5,000円 × 10個)
  - 不人気商品: りんご (200円 × 50個が売れたが、利益率が低い)

- **月次トレンド**:
  - 1月から4月にかけて売上が急増し、特に4月は季節的に最多の売上を記録。

- **利益率**: 
  - 総利益: 約12,000,000円
  - 苦戦している商品での原価が高い（例: 一部の衣服商品）

### 結論
このような分析を実施することで、今後のマーケティング戦略や在庫管理の方針を策定するための重要なインサイトを得ることができます。データを基に、売上向上のための具体的な施策や予算配分を行うことが可能です。具体的なデータ集計を行うためには、デー

In [6]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0                        売上データの分析を行い、以下の観点から傾向を検討します。
1                                                    
2                                    ### 1. **売上の総額**
3            まず、全体の売上総額を計算します。売上総額は、「単価 × 数量」で求められます。
4                                                    
5                                ### 2. **カテゴリ別売上分析**
6   売上データのカテゴリ（食品、衣服、日用品など）ごとに売上を集計し、どのカテゴリが最も売上を生...
7                                                    
8                               ### 3. **商品のパフォーマンス**
9   各商品の売上貢献度を集計し、特に売上が高い商品や、逆に不人気な商品の分析を行います。ここでは...
10                                                   
11                              ### 4. **日付別売上のトレンド**
12  売上が行われた日付に基づき、月次・週次での売上のトレンドを調べ、季節性や特定のイベントが影響...
13                                                   
14                                    ### 5. **利益分析**
15  商品の利益を算出し、利益率（利益 ／ 売上）や全体の利益額を分析します。これは企業の健全性を...
16                                                   
17                          

In [7]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [8]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
